In [2]:
from collections import defaultdict
import numpy as np

In [3]:
def read(filename):
    users = set()
    user_directed_graph =  defaultdict(lambda: defaultdict(float))

    ads = set()
    ads_directed_graph = defaultdict(lambda: defaultdict(float))

    input = open(filename, 'r')
    num_entries = int(input.readline())

    for i in range(num_entries):
        unformatted = input.readline()[:-1].split(',')
        score = float(unformatted[-1])
        unformatted.pop()
        cur_user, cur_ad = [int(entry) for entry in unformatted]
        users.add(cur_user)
        user_directed_graph[cur_user][cur_ad] = score

        ads.add(cur_ad)
        ads_directed_graph[cur_ad][cur_user] = score

    predict_user, predict_ad = [int(entry) for entry in input.readline().split(',')]
    
    return users, user_directed_graph, ads, ads_directed_graph, predict_user, predict_ad



In [26]:
def simple_simrank(users, user_directed_graph, ads, ads_directed_graph):
    # note that this runs simrank with the partial sums memoization trick!
    C1 = C2 = 0.8
    similarity_user = defaultdict(lambda: defaultdict(float))
    similarity_ads = defaultdict(lambda: defaultdict(float))

    for u in users:
        similarity_user[u][u] = 1.0
    
    for a in ads:
        similarity_ads[a][a] = 1.0

    partial_user = defaultdict(lambda: defaultdict(float))
    partial_ads = defaultdict(lambda: defaultdict(float))


    for iter in range(10):
        for u in users:
            for a in ads:
                temp = 0.0
                for u_ads in user_directed_graph[u]:
                    temp += similarity_ads[u_ads][a]
                partial_user[u][a] = temp
        
        list_users = list(users)
        for i in range(len(list_users)):
            for j in range(i + 1, len(list_users)):
                temp = 0.0
                u1, u2 = list_users[i], list_users[j]
                for a in user_directed_graph[u2]:
                    temp += partial_user[u1][a]
                similarity_user[u1][u2] = similarity_user[u2][u1] =  C1 / (len(user_directed_graph[u1])*len(user_directed_graph[u2])) * temp
    
        for a in ads:
            for u in users:
                temp = 0.0
                for a_user in ads_directed_graph[a]:
                    temp += similarity_user[a_user][u]
                partial_ads[a][u] = temp
        
        list_ads = list(ads)
        for i in range(len(list_ads)):
            for j in range(i+1, len(list_ads)):
                temp = 0.0
                a1, a2 = list_ads[i], list_ads[j]
                for k in ads_directed_graph[a2]:
                    temp += partial_ads[a1][k]
                similarity_ads[a1][a2] = similarity_ads[a2][a1] = C2 / (len(ads_directed_graph[a1])*len(ads_directed_graph[a2])) * temp
            
    return similarity_user, similarity_ads
                

In [5]:
def geometric_evidence(similarity_ads, ads_directed_graph, ads, similarity_users, user_directed_graph, users):
    for _ in range(1):
        copy_ads = list(ads)
        for i in range(len(copy_ads)):
            for j in range(i + 1, len(copy_ads)):
                a1, a2 = copy_ads[i], copy_ads[j]
                a1_neighbors, a2_neighbors = ads_directed_graph[a1].keys(), ads_directed_graph[a2].keys()

                evidence_sum = 0.0
                # starting adn ending 1 later to account for offset
                for power in range(1, len(set(a1_neighbors).intersection(set(a2_neighbors))) + 1):
                    evidence_sum += (1/2)**power
                
                similarity_ads[a1][a2] *= evidence_sum
                similarity_ads[a2][a1] = similarity_ads[a1][a2]
        
        copy_users = list(users)
        for i in range(len(copy_users)):
            for j in range(i + 1, len(copy_users)):
                u1, u2 = copy_users[i], copy_users[j]
                if u1 == 1 and u2 == 4:
                    print('hello')
                u1_neighbors, u2_neighbors = user_directed_graph[u1].keys(), user_directed_graph[u2].keys()

                evidence_sum = 0.0
                # starting adn ending 1 later to account for offset
                for power in range(1, len(set(u1_neighbors).intersection(set(u2_neighbors))) + 1):
                    evidence_sum += (1/2)**power
                
                similarity_users[u1][u2] *= evidence_sum
                similarity_users[u2][u1] = similarity_users[u1][u2]
        
    
    return similarity_users, similarity_ads

In [6]:
def exponential_evidence(similarity_ads, ads_directed_graph, ads, similarity_users, user_directed_graph, users):
    for _ in range(1):
        copy_ads = list(ads)
        for i in range(len(copy_ads)):
            for j in range(i + 1, len(copy_ads)):
                a1, a2 = copy_ads[i], copy_ads[j]
                a1_neighbors, a2_neighbors = ads_directed_graph[a1].keys(), ads_directed_graph[a2].keys()

                evidence_sum = 1-np.exp(-len(set(a1_neighbors).intersection(set(a2_neighbors))))
                
                similarity_ads[a1][a2] *= evidence_sum
                similarity_ads[a2][a1] = similarity_ads[a1][a2]
        
        copy_users = list(users)
        for i in range(len(copy_users)):
            for j in range(i + 1, len(copy_users)):
                u1, u2 = copy_users[i], copy_users[j]
                u1_neighbors, u2_neighbors = user_directed_graph[u1].keys(), user_directed_graph[u2].keys()

                evidence_sum = 1-np.exp(-len(set(u1_neighbors).intersection(set(u2_neighbors))))
                # starting adn ending 1 later to account for offset
                
                similarity_users[u1][u2] *= evidence_sum
                similarity_users[u2][u1] = similarity_users[u1][u2]
    
    
    return similarity_users, similarity_ads

In [7]:
def fetch_top3(to_predict, similarity):
    res = []
    temp = similarity[to_predict].items()

    value_key = {}
    for k,v in temp:
        if k != to_predict:
            if v not in list(value_key.keys()):
                value_key[v] = k
                res.append((k,v))
            else:
                if k < value_key[v]:
                    res.remove((value_key[v], v))
                    value_key[v] = k
                    res.append((k,v))
                    
    res = sorted(res, reverse=True, key= lambda x: (x[1], -x[0]))

    return res[:3]

In [8]:
def writeOutput(users, ads):
    f = open("final_output.txt", "a")
    final_str = ''
    for k,v in users:
        final_str += f'{k},'
    final_str = final_str[:-1]
    final_str += '\n'
    f.write(final_str)

    final_str = ''
    for k,v in ads:
        final_str += f'{k},'
    final_str = final_str[:-1]
    final_str += '\n'
    f.write(final_str)



In [116]:
users, user_directed_graph, ads, ads_directed_graph, predict_user, predict_ad = read(filename='input_b.txt')
similarity_user, similarity_ads = simple_simrank(users, user_directed_graph, ads, ads_directed_graph)

top3_users = fetch_top3(predict_user, similarity_user)
top3_ads = fetch_top3(predict_ad, similarity_ads)
# top3_users, top3_ads
writeOutput(top3_users, top3_ads)

In [117]:
users, user_directed_graph, ads, ads_directed_graph, predict_user, predict_ad = read(filename='input_b.txt')
similarity_user, similarity_ads = simple_simrank(users, user_directed_graph, ads, ads_directed_graph)
similarity_user, similarity_ads = geometric_evidence(similarity_ads, ads_directed_graph, ads, similarity_user, user_directed_graph, users)

top3_users = fetch_top3(predict_user, similarity_user)
top3_ads = fetch_top3(predict_ad, similarity_ads)
# top3_users, top3_ads
writeOutput(top3_users, top3_ads)

In [118]:
users, user_directed_graph, ads, ads_directed_graph, predict_user, predict_ad = read(filename='input_b.txt')
similarity_user, similarity_ads = simple_simrank(users, user_directed_graph, ads, ads_directed_graph)
similarity_user, similarity_ads = exponential_evidence(similarity_ads, ads_directed_graph, ads, similarity_user, user_directed_graph, users)

top3_users = fetch_top3(predict_user, similarity_user)
top3_ads = fetch_top3(predict_ad, similarity_ads)
# top3_users, top3_ads
writeOutput(top3_users, top3_ads)

In [76]:
def task3_breaking_simrank(user_directed_graph, ads_directed_graph, users, ads):
    dummy_ads = [10001, 10002, 10003]
    # for d_ad in dummy_ads:
    #     for u in users:
    #         ads.add(d_ad)
    #         temp = 0
            
    #         for neighbor in user_directed_graph[u]:
    #             temp = max(temp, user_directed_graph[u][neighbor]) + 2
    #         user_directed_graph[u][d_ad] = temp 
    #         ads_directed_graph[d_ad][u] = temp 
    # ads.add(10001)
    # ads_directed_graph[10001][24481] = 6.0
    # user_directed_graph[24481][10001] = 6.0
    for d_ad in dummy_ads:
        ads.add(d_ad)
        for dup_ad in ads:
            for associated_user in ads_directed_graph[dup_ad]:
                ads_directed_graph[d_ad][associated_user] = ads_directed_graph[dup_ad][associated_user]
                user_directed_graph[associated_user][d_ad] = user_directed_graph[associated_user][dup_ad]



In [77]:
users, user_directed_graph, ads, ads_directed_graph, predict_user, predict_ad = read(filename='input_b.txt')
# print("before: " + str(ads_directed_graph[10001]))
task3_breaking_simrank(user_directed_graph, ads_directed_graph, users, ads)
# print("after: " + str(ads_directed_graph[10001]))

similarity_user, similarity_ads = simple_simrank(users, user_directed_graph, ads, ads_directed_graph)
# similarity_user, similarity_ads = geometric_evidence(similarity_ads, ads_directed_graph, ads, similarity_user, user_directed_graph, users)

top3_users = fetch_top3(predict_user, similarity_user)
top3_ads = fetch_top3(predict_ad, similarity_ads)
top3_users, top3_ads


defaultdict(<class 'float'>, {24481: 3.0, 579: 1.0, 76584: 2.0, 896: 3.0, 3374: 1.0, 661: 2.0, 5786: 1.0})


([(24481, 0.31884531578582276),
  (76584, 0.26922201901093906),
  (5786, 0.24932404233538857)],
 [(0, 0.5330713830366517), (5, 0.433789899997336), (37, 0.43378989999733597)])

In [74]:
ads_directed_graph[0]

defaultdict(float, {24481: 5.0})

In [75]:
similarity_ads[6]
# similarity_ads[10001]

defaultdict(float,
            {6: 1.0,
             0: 0.5330713830366517,
             1: 0.4257758172952364,
             2: 0.3841491584776782,
             3: 0.4335146878525801,
             4: 0.5330713830366517,
             5: 0.433789899997336,
             7: 0.3837363402605444,
             8: 0.4335146878525801,
             9: 0.38414915847767817,
             10: 0.5330713830366517,
             11: 0.5330713830366517,
             12: 0.38394274936911127,
             13: 0.3837363402605444,
             14: 0.33251228661160714,
             15: 0.5330713830366517,
             16: 0.4335146878525801,
             17: 0.43208363039088604,
             19: 0.5330713830366517,
             20: 0.5330713830366517,
             21: 0.5330713830366517,
             22: 0.3837363402605444,
             23: 0.4257758172952364,
             25: 0.38414915847767817,
             26: 0.32951063473136766,
             27: 0.5330713830366517,
             28: 0.5330713830366517,
  

In [1]:
def simrank_plus_plus(user_directed_graph, ad_directed_links, users, ads):
    similarity_user = defaultdict(lambda: defaultdict(float))
    similarity_ads = defaultdict(lambda: defaultdict(float))

    # computing spread
    user_spread = {}
    for u in users:
        weights = []
        for k,v in user_directed_graph[u].items():
            weights.append(v)
        weights = np.array(weights)
        user_spread[u] = np.exp(-weights.var())

    ad_spread = {}
    for a in ads:
        weights = []
        for k,v in ads_directed_graph[a].items():
            weights.append(v)
        weights = np.array(weights)
        ad_spread[a] = np.exp(-weights.var())
    
    
        

    return similarity_user, similarity_ads


IndentationError: expected an indented block (1091784693.py, line 1)

In [ ]:

ad_spread = {}
user_spread = {}

def spread_func(i, graph, ads):
    if ads:
        if i in list(ad_spread.keys()):
            return ad_spread[i]
        weights = []
        for k,v in graph[i].items():
            weights.append(v)
        weights = np.array(weights)
        ad_spread[i] = np.exp(-weights.var())
        return ad_spread[i]
    else:
        if i in list(user_spread.keys()):
            return user_spread[i]
        weights = []
        for k,v in graph[i].items():
            weights.append(v)
        weights = np.array(weights)
        user_spread[i] = np.exp(-weights.var())
        return user_spread[i]